Dear Colleagues,

In this Kernel I am glad to present you with an alternative view on the features analysis and it's impact on the target. Firstly a little bit information about the features of this dataset:


* Diabetes - 0 = No, 1 = Yes
* Sex - Gender of patient Male = 1, Female =0
* Age - Age of patient
* High_blood_pressure - 0 = No, 1 = Yes
* Smoking - 0 = No, 1 = Yes
* Anaemia - 0 = No, 1 = Yes
* Time - time of patient's observation before his/her removal or/and death in days
* DEATH_EVENT - 0 = No, 1 = Yes

Other features will be considered  in the process of their study.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt    
import seaborn as sns 

def basic_analysis(df1, df2):
    '''the function compares the average values of  2 dataframes'''
    b = pd.DataFrame()
    b['First df_mean'] = round(df1.mean(),2)
    b['Second df_mean'] = round(df2.mean(),2)
    c = (b['First df_mean']/b['Second df_mean'])
    if [c<=1]:
        b['Variation, %'] = round((1-((b['First df_mean']/b['Second df_mean'])))*100)
    else:
        b['Variation, %'] = round(((b['First df_mean']/b['Second df_mean'])-1)*100)
        
    b['Influence'] = np.where(abs(b['Variation, %']) <= 9, "feature's effect on the target is not defined", 
                              "feature value affects the target")

    return b

def base_analysis(df1, df2, df3):
    '''the function compares the average values of  3  dataframes'''
    b = pd.DataFrame()
    b['First_df'] = round(df1.mean(),2)
    b['Second_df'] = round(df2.mean(),2)
    b['Third_df'] = round(df3.mean(),2)
    c = (b['First_df']/b['Third_df'])
    if [c<=1]:
        b['Variation, %'] = round((1-((b['First_df']/b['Third_df'])))*100)
    else:
        b['Variation, %'] = round(((b['First_df']/b['Third_df'])-1)*100)
        
    b['Influence'] = np.where(abs(b['Variation, %']) <= 9, "feature's value effect on survival is not defined", 
                              "feature value affects survival")

    return b


def basic_details(df):
    '''returns dataframe with feature distribution: missing values, unique values, its type'''

    b = pd.DataFrame()
    b['Missing value, %'] = round(df.isnull().sum()/df.shape[0]*100)
    b['N unique value'] = df.nunique()
    b['dtype'] = df.dtypes
    return b


In [ ]:
## let's import dataset as df
df = pd.read_csv('/kaggle/input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')

In [ ]:
# and checking for general info as for dataset....
df.head()

....and some statistical information....

In [ ]:
df.describe().T

... and some info of feature distribution...

In [ ]:
basic_details(df)

# EDA

Let's analyze how each indicator affects the target

first of all for quick analysis let's split dataset into 2 parts by target feature (Survived and Not survived patients) ....

In [ ]:
survived = df.drop(df[df['DEATH_EVENT'] != 0].index)
not_survived = df.drop(df[df['DEATH_EVENT'] != 1].index)

....and compare them: how difference between them affects on target

In [ ]:
import matplotlib.pyplot as plt
features = df.drop(['DEATH_EVENT', 'time', 'anaemia', 'diabetes', 'high_blood_pressure', 'sex', 'smoking'], axis=1).columns
for i in features:
    sns.boxplot(x="DEATH_EVENT",y=i,data=df)
    plt.title(i+" by "+"DEATH_EVENT")
    plt.show()

In [ ]:
basic_analysis(survived,not_survived)

What we see here - for example:
 - the presence or absence of diabetes does not in any way affect the mortality rate or its impact is still difficult to assess based on these data
 - the gender of the patient also does not affect the mortality rate
 - the number of deaths in patients diagnosed with "hypertension" (high blood pressure) is 20% more than in the subject who did not have this diagnosis

**Let me give you a full list of conclusions at the end of this investigation?**

Also I want to **remove the time variable** as it correlates highly with the target variable. Hopefully the performance will not be affected. If you have reasons why it is not necessary to do this, please let me know about it.

In [ ]:
## let's drop this featute coz it's time of patient's observation before his/her removal or/and death
df=df.drop('time', axis=1)

To obtain complete information on the influence of all indicators on each other, we can in the same way divide the data frame into 2 or more parts and compare these parts with each other for similarities and differences.

For example, let's divide dataframe into 2 parts **by "smoking" feature**:

In [ ]:
smoking = df[df['smoking'] == 1]
non_smoking = df[df['smoking'] == 0]
basic_analysis(smoking,non_smoking)

Several conclusions based on the data obtained:

  - nonsmoking patients - approximately equal number of both men and women
  - smoking patients - mostly men
  - the total level of platelets in the blood is on average the same in the group of smokers and non-smokers

Or we can for example divide dataframe into 3 parts (for example by "platelets" level) and compare them to each other:

In [ ]:
df['platelets_cat'] = df['platelets'].map(lambda x: 0 if x <= 150000 
                                                            else (1 if 450000 >= x > 150000 
                                                                  else 2))
low_platelets = df[df['platelets_cat'] == 0]
norm_platelets = df[df['platelets_cat'] == 1]
high_platelets = df[df['platelets_cat'] == 2]  

base_analysis(low_platelets,norm_platelets,high_platelets)

General conclusions:

1.	The average age of the surviving patients is 12% lower than the age of those who did not survive: the younger ones survived.

2.	Cases of anaemia(a condition characterized by a decrease in the number of red blood cells and a decrease in the hemoglobin content per unit of blood volume) occur in the group of surviving patients on average 15% less frequently than in the group of patients who did not survive: the presence of anemia worsens the prognosis for patient survival ... Patients diagnosed with anemia in this study:

 - 	the CPK level is 50% lower compared to patients without this diagnosis
 - 	11% more likely to be diagnosed with hypertension
 - 	8% higher creatinine
 
3.	The level of creatine kinase (creatine phosphokinase, CPK is an enzyme due to which muscle contractions are provided with energy, due to the conversion of creatine into creatine phosphate, the norm is from 25 to 200 U / L, U / l. Exceeding these values is possible with hypothyroidism, myocarditis, myocardial infarction, myocardial dystrophy, dermatomy , polymyositis, tumors, trauma, burns and under the influence of certain drugs) in the group of survivors is on average 20% lower than in the group of patients who did not survive: an increase in the CPK level leads to an increase in patient mortality.

4.	The diagnosis of diabetes occurs in both groups of patients in the same proportion. In the framework of this study, this indicator can be neglected, since it is not possible to assess the effect of this indicator on mortality. In addition: The diagnosis of diabetes occurs 2 times more often in the group of patients with an increased level of platelets in the blood (above 550 thousand U / μL) compared with the indicator in the group of patients with a low (average value of 115 thousand IU / μL) platelet level.

5.	Ejection fraction (the volume fraction of blood in% ejected from the left ventricle with each contraction of the heart muscle. The norm is from 50 to 70%, below 40% indicates cardiomyopathy, above 75% - about hypertrophied cardiomyopathy): in the group of survivors this indicator is higher by 20%: A decrease in ejection function leads to an increase in patient mortality. Within the framework of this study, it was determined: the higher the level of platelets in the patient's blood, the higher the level of ejection fraction.

6.	The diagnosis of "hypertension" (high blood pressure) is observed more often by 25% in the group of deceased patients: an increased level of blood pressure leads to an increase in the death rate. In addition to the above:

 - 	hypertensive patients are 9% more likely to suffer from anemia
 - 	their CPK level is 23% lower
 - 	the diagnosis of "hypertension" in groups of patients with increased and decreased levels of platelets is 2 times less common than in the group of patients with normal levels of platelets in the blood.
 
7.	Platelets (blood elements that are responsible for blood clotting. The average norm is 200-400 thousand units / μl): on average, their level is the same for both groups and is on average within the medical norm. In addition:

 - 	in the group of patients with an increased platelet count (over 550 thousand units / μl), the diagnosis of anemia is 30% less common
 - 	in the same group, the age of patients is 7% higher in relation to other patients
 - 	in the group of patients with a low platelet level (an average of 115 thousand units / μl), the CPK level is on average 730 units / l (almost four times the norm in 200 units / l), in the group with an average (265 thousand units / μl) - 570 units / l, in the group with increased (average value 550 thousand units / μl) - 450 units / l (that is, the lower the CPK level, the higher the platelet level)
 - 	the mortality rate of patients with high and low blood platelet levels is 25% higher than in the group of patients with normal platelet levels.
 
8.	Creatinine (determined by the volume of muscle mass, participates in the energy metabolism of body tissues, the norm is on average 0.5 - 1.2 mg / dl, the creatinine level increases with dehydration, muscle damage, physical activity, renal failure): its level is higher on average 55% in the group of non-survivors: an increased level of creatinine in the blood leads to an increase in mortality.

9.	Sodium in the blood (a vital trace element that is necessary for the transmission of impulses in the nervous system and muscle contractions, the norm is 136-145 mmol / l, the level of sodium in the blood below 120 mmol / l is life threatening): on average, the same in both groups, its level does not threaten the life of patients.

10.	The distribution of patients by gender is approximately the same. Within the framework of this study, we assume that this indicator did not affect the mortality rate of patients. However, in this study:

 - 	in men there is 20% less diagnosis of anaemia
 - 	men have 34% higher CPK level
 - 	diabetes is 44% less common in men
 - 	ejection fraction in men is 10% lower
 - 	high blood pressure affects 26% fewer men
 - 	platelet count in men is 10% lower

11.	The number of smokers is also about the same in the groups. Within the framework of this study, we assume that this indicator did not affect the mortality rate of patients.

12.	96% of smokers are men, non-smokers are equally.


Выводы:
1. Средний возраст выживших пациентов  на 12% ниже возраста тех, кто не выжил: выжили более молодые.

2. Случаи анемии/малокровия (состояние, для которого характерно уменьшение количества эритроцитов и снижение содержания гемоглобина в единице объема крови) встречаются в группе выживших пациентов в среднем на 15% реже, чем в группе пациентов, кто не выжил: наличие анемии ухудшает прогноз по выживаемости пациента. Пациентам с диагнозом анемия в рамках данного исследования:
 - уровень КФК на 50% ниже по сравнению с пациентами без данного диагноза
 - на 11% чаще ставят диагноз "гипертония"
 - на 8% выше показатель креатинина

3. Уровень креатинкиназы (креатинфосфокиназа, КФК - фермент, благодаря которому мышечные сокращения обеспечиваются энергией, вследствие превращения креатина в креатинфосфат, норма от 25 до 200 ед/л, U/l. Превышение этих значений возможно  при гипотиреозе, миокардите, инфаркте миокарда, миокардиодистрофии, дерматомиозите, полимиозите, опухолях, травмах, ожогах и под влиянием некоторых препаратов) в группе выживших в среднем на 20% ниже по сравнению с показателями в группе не выживших пациентов: увеличение уровня КФК ведет к увеличению смертности пациентов. 


4. Диагноз "диабет" встречается в обеих группах пациентов пропорционально одинаково. В рамках данного исследования данным показателем можно пренебречь, поскольку нет возможности оценить влияние данного показателя на смертность. В дополнение: 
 - диагноз диабет встречается в 2 раза чаще в группе пациентов с повышенным уровнем тромбоцитов в крови (выше 550 тыс ЕД/мкл) по сравнению с показателем в группе пациентов с пониженным (средний показатель 115 тыс ЕД/мкл) уровнем тромбоцитов. 

5. Фракция выброса (объемная доля крови в %, выбрасываемой из левого желудочка при каждом сокращении сердечной мышцы. Норма - от 50 до 70%, ниже 40% говорит о кардиомиопатии, выше 75% - о гипертрофированной кардиомиопатии): в группе выживших этот показатель выше на 20%: снижение функции выброса ведет к увеличению показателя смертности пациентов. В рамках данного исследования определено: чем выше уровень тромбоцитов в крови пациента - тем выше уровень фракции выброса. 

6. Диагноз "гипертония" наблюдается чаще на 25% в группе умерших пациентов: повышенный уровень кровяного давления ведет к увеличению показателя смертности. В дополнение к сказанному:
 - гипертоники на 9% чаще страдают от анемии
 - уровень КФК у них на 23% ниже
 - диагноз "гипертония" в группах пациентов с повышенным и пониженным уровнем тромбоцитов встречается в 2 раза реже по сравнению с группой пациентов с нормальным уровнем тромбоцитов в крови.

7. Тромбоциты (элементы крови, которые отвечают за свёртываемость крови. Норма в среднем 200-400 тыс ЕД/мкл): в среднем их уровень одинаков для обоих групп и находится в среднем в пределах медицинской нормы.  В дополнение:
 - в группе пациентов с повышенным уровнем тромбоцитов (свыше 550 тыс ЕД/мкл) на 30% реже встречается диагноз "анемия"
 - в этой же группе возраст пациентов на 7% выше по отношению к другим пациентам
 - в группе пациентов с пониженным уровнем тромбоцитов (средний показатель 115 тыс ЕД/мкл) уровень КФК составляет в среднем 730 ед/л (почти четырехкратное превышение нормы в 200 ед/л), в группе со средним (265 тыс ЕД/мкл) - 570 ед/л, в группе с повышенным (средний показатель 550 тыс ЕД/мкл) - 450 ед/л (то есть чем ниже уровень КФК - тем выше уровень тромбоцитов)
 - уровень смертности пациентов с повышенным и пониженным уровнем тромбоцитов в крови выше на 25% по сравнению с группой пациентов с уровнем тромбоцитов в пределах нормы.

8. Креатинин (определяется объемом мышечной массы, участвует в энергетическом обмене тканей организма, норма в среднем 0,5 - 1,2 мг/дл, уровень креатинина возрастает при обезвоживании организма, поражении мышц, физической нагрузке, почечной недостаточности): его уровень выше в среднем на 55% в группе не выживших пациентов: : повышенный уровень креатинина в крови ведет к увеличению показателя смертности.

9. Натрий в крови (жизненно важный микроэлемент, который необходим для передачи импульсов в нервной системе и мышечных сокращений, норма 136-145 ммоль/л, уровень натрия в крови ниже 120 ммоль/л угрожает жизни): в среднем одинаков в обеих группах, его уровень не угрожает жизни пациентов.

10. Распределение пациентов по половому признаку примерно одинаково. В рамках данного исследования принимаем, что этот показатель не повлиял на уровень смертности пациентов. Однако в рамках данного исследования:
 - у мужчин на 20% меньше встречается диагноз "малокровие/анемия"
 - у мужчин на 34% выше уровень КФК
 - у мужчин на 44% реже встречается диабет
 - фракция выброса у мужчин на 10% ниже
 - высоким давлением страдают на 26% меньше мужчин
 - уровень тромбоцитов у мужчин на 10% ниже

11. Число курильщиков также примерно одинаково в группах. В рамках данного исследования принимаем, что этот показатель не повлиял на уровень смертности пациентов.

12. 96% курильщиков - мужчины, некурящих - поровну.

**...and XGBOOST modelling**

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


X = df.drop('DEATH_EVENT', axis=1)
y = df.DEATH_EVENT

x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=10,shuffle=True, stratify=y)

d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_valid, label=y_valid)


params = {
        'objective':'binary:logistic',
        'max_depth':4,
        'learning_rate':0.3,
        'eval_metric':'auc',
        'min_child_weight':1,
        'subsample':0.85,
        'colsample_bytree':0.75,
        'seed':29,
        'reg_lambda':2.8,
        'reg_alpha':0,
        'gamma':0,
        'scale_pos_weight':1,
        'nthread':-1
}

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

nrounds=10000  
model = xgb.train(params, d_train, nrounds, watchlist, early_stopping_rounds=600,   
  
                           maximize=True, verbose_eval=10)

Final model score:

In [ ]:
accuracy = pd.DataFrame()
accuracy['predict'] = model.predict(d_valid)
accuracy['predict'] = accuracy['predict'].apply(lambda x: 1 if x>0.8 else 0)
accuracy_score(y_valid, accuracy['predict'])

In [ ]:
xgb.plot_importance(model)

matrix correlation...

In [ ]:
def spearman(frame, features):
    spr = pd.DataFrame()
    spr['feature'] = features
    spr['spearman'] = [frame[f].corr(frame['DEATH_EVENT'], 'spearman') for f in features]
    spr = spr.sort_values('spearman')
    plt.figure(figsize=(6, 0.25*len(features)))
    sns.barplot(data=spr, y='feature', x='spearman', orient='h')
    
features = df.columns
spearman(df, features)

Feature importance graph confirms our earlier conclusions.....

...and decision tree for our Model:

In [ ]:
xgb.to_graphviz(model)

Thanks for your attention and pls revert with your comments and upvotes